In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('blogtext.csv').sample(40000)
df.head()

,id,gender,age,topic,sign,date,text
12240,1926378,female,24,indUnk,Aries,"09,June,2004","So, Michael Moore's new film 'Fahren..."
170857,3027884,female,33,Non-Profit,Aquarius,"03,May,2004",I had to go to Walmart this m...
253195,3849840,male,16,Student,Virgo,"09,July,2004",Aww..i am leaving for Fl. tomorrow...do...
278378,1169141,male,33,Arts,Capricorn,"01,June,2003","BBQ and overkill oH MAN, I jus..."
291205,2929974,female,36,Publishing,Leo,"26,February,2004",Every year about this time I start thinking...


In [25]:
X=df['text'].astype(str)
Y=pd.get_dummies(df['gender'])

In [26]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.30, random_state = 100)
X_train, X_valid, Y_train, Y_valid = train_test_split(X_train,Y_train, test_size = 0.30, random_state = 100)
print(X_train.shape,Y_train.shape)
print(X_valid.shape,Y_valid.shape)
print(X_test.shape,Y_test.shape)

(19600,) (19600, 2)
(8400,) (8400, 2)
(12000,) (12000, 2)


In [50]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
train_tokens = X_train.map(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:64])
test_tokens = X_test.apply(lambda t: ['[CLS]'] + tokenizer.tokenize(t)[:64])
train_tokens_ids = pd.Series(train_tokens).apply(tokenizer.convert_tokens_to_ids)
test_tokens_ids = pd.Series(test_tokens).apply(tokenizer.convert_tokens_to_ids)
train_tokens_ids.shape

(19600,)

In [60]:
#Do I need this step?
import keras
from keras.preprocessing.sequence import pad_sequences
train_tokens_ids = pad_sequences(train_tokens_ids, maxlen=64, truncating="post", padding="post", dtype="int")
test_tokens_ids = pad_sequences(test_tokens_ids, maxlen=64, truncating="post", padding="post", dtype="int")

In [62]:
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
BATCH_SIZE = 64
test_y = np.array(Y_test)
train_y = np.array(Y_train)
train_tokens_tensor = torch.LongTensor(train_tokens_ids)
train_y_tensor = torch.from_numpy(train_y)
test_tokens_tensor = torch.LongTensor(test_tokens_ids)
test_y_tensor = torch.from_numpy(test_y)
train_dataset = TensorDataset(train_tokens_tensor, train_y_tensor)
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=BATCH_SIZE)
test_dataset = TensorDataset(test_tokens_tensor, test_y_tensor)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=BATCH_SIZE)

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.is_available()
train_y_tensor
from transformers import (
    WEIGHTS_NAME,
    AdamW,
    BertConfig,
    BertForMultipleChoice,
    BertTokenizer,
    RobertaConfig,
    RobertaForMultipleChoice,
    RobertaTokenizer,
    XLNetConfig,
    XLNetForMultipleChoice,
    XLNetTokenizer,
    get_linear_schedule_with_warmup
    )

In [66]:
from transformers import AdamW, BertForSequenceClassification, BertModel
EPOCHS=3
con = BertConfig(vocab_size_or_config_json_file=30522)
bert_clf = BertForSequenceClassification(config = con)
optimizer = AdamW(bert_clf.parameters(), lr=3e-6)
bert_clf.train()
for epoch_num in range(EPOCHS):
    for step_num, batch_data in enumerate(train_dataloader):
        token_ids, labels = [t for t in batch_data]
        labels = torch.autograd.Variable(labels)
        probas = bert_clf(token_ids.to(torch.int64))
        loss_func = nn.CrossEntropyLoss()
        batch_loss = loss_func(probas, labels)
        bert_clf.zero_grad()
        batch_loss.backward()
        optimizer.step()

AttributeError: 'tuple' object has no attribute 'log_softmax'

In [67]:
# My problem seems to be a shaping problem. Googling and looking at the PyTorch forums hasn't really helped.
probas
# probas.shape

(tensor([[-0.0231, -0.0355],
         [ 0.0405, -0.1860],
         [ 0.4154,  0.0209],
         [ 0.3506,  0.1075],
         [ 0.3131, -0.0650],
         [ 0.0087,  0.0698],
         [ 0.2654, -0.0740],
         [ 0.0859, -0.0813],
         [ 0.1231, -0.1525],
         [ 0.3443,  0.1602],
         [ 0.0948,  0.1089],
         [ 0.2314, -0.2188],
         [ 0.0339, -0.0992],
         [ 0.1025, -0.1835],
         [ 0.3428,  0.0027],
         [ 0.0770, -0.2052],
         [ 0.4102,  0.0588],
         [ 0.1848, -0.1553],
         [ 0.1020, -0.3258],
         [ 0.1746, -0.1525],
         [ 0.3763, -0.0120],
         [ 0.4577, -0.1242],
         [ 0.3115, -0.0433],
         [ 0.2976, -0.0897],
         [ 0.0461, -0.0145],
         [ 0.1704, -0.1950],
         [ 0.1516, -0.2341],
         [-0.0765,  0.0220],
         [ 0.1270, -0.2870],
         [ 0.2282,  0.0089],
         [ 0.1738, -0.2086],
         [ 0.1883, -0.2205],
         [ 0.1445,  0.0703],
         [ 0.2909, -0.2309],
         [ 0.1